In [4]:
from torchvision.io import read_image
import torch.nn as nn
import torch.nn.functional as F
import torch 
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from utils import show_image
import os
import cv2
import matplotlib.pyplot as plt
import time
from turtle import forward
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

from dataset import NMDataset
from utils import get_train_dev_test_data
from training import Training,validate
from modeling import encoders,decoders,OUR_model


In [5]:
transformations = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([1024,1024]),
    transforms.ToTensor(),
    # transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])


# create dataset 
print("__________Create Dataset___________")
train_set = NMDataset('train',transformations)
valid_set = NMDataset('val',transformations)
test_set = NMDataset('test',transformations)

train_loader = DataLoader(train_set, batch_size = 3, shuffle = True)
valid_loader = DataLoader(valid_set, batch_size = 3, shuffle = True)
test_loader = DataLoader(test_set, batch_size = 3, shuffle = True)



__________Create Dataset___________


In [ ]:
# create view_data
for data, label in train_loader:
  view_data = data[0:3]
  break


# model prepare
model = OUR_model().cuda()
Loss1 = nn.CrossEntropyLoss()
Loss2 = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


print("____________Training_____________")
num_epochs = 20
import tqdm
for epoch in range(num_epochs):
    start_time = time.perf_counter()

    loss,loss1,loss2= Training(model,Loss1,Loss2,optimizer, train_loader)
    val_loss,f1 = validate(model,Loss1,Loss2, valid_loader)
    end_time = time.perf_counter()


    ckpt_dict = {
    'epoch': epoch + 1,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict() 
    }

    torch.save(ckpt_dict, r'ckpt\best_model_1102.pth.tar')

    if epoch == 0:
        best_rmse = loss
    elif loss < best_rmse:
        best_rmse = loss
        torch.save(ckpt_dict, r'ckpt\best_model_1102.pth.tar')



    print('Epoch : {}, loss:{:.4f}, loss1:{:.4f},loss2:{:.4f},val_loss:{:.4f}, test_f1:{:.2f} {:.2f} seconds.'.format(epoch+1, loss, loss1,loss2,val_loss,f1,end_time-start_time))

    if epoch % 5 == 0:
        output_decoders = []
        conf_level,output_encoder,output_decoder = model(view_data.cuda())  
        output_decoders.append(output_decoder)


        f, axarr = plt.subplots(2,3,figsize= (10,6))
        n = 0
        for out_data, test_data in zip(output_decoder,view_data):
            out_data = out_data.permute(1,2,0).cpu().detach().numpy()   
            test_data  = test_data.permute(1,2,0).cpu().numpy() 


            axarr[0,n].imshow((out_data * 255).astype(np.uint8))
            axarr[1,n].imshow((test_data * 255).astype(np.uint8))
            n+=1
        plt.show()


In [ ]:
path="D:/NewEra/M"
A=[]

for i in os.listdir("D:/NewEra/M"):
    file=i.split("_")

    if len(i.split("_"))==3:
        A.append(file[0]+"_1")
    else:
        A.append(file[0])
B=list(set(A))
print(len(B))
train_folder = B[:28]
val_folder = B[28:35]
test_folder = B[35:]


In [ ]:


train_file=[]
train_folders=[]
train_label=[]
test_file=[]
test_folders=[]
test_label=[]
val_file=[]
val_folders=[]
val_label=[]
no=[]
for file in os.listdir(path):

        if len(file.split("_"))==3:
            folder_name = file.split("_")[0]+"_1"
        else:
            folder_name = file.split("_")[0]
        
        if folder_name in val_folder:
            val_file.append([folder_name,file.split("_")[-1][:-4],"M"])
            # val_folders.append(folder_name)
            # val_label.append("No")
        elif folder_name in test_folder:
            test_file.append([folder_name,file.split("_")[-1][:-4],"M"])
            # test_folders.append(folder_name)
            # test_label.append("No")
        elif folder_name in train_folder:
            train_file.append([folder_name,file.split("_")[-1][:-4],"M"])
            # train_folders.append(folder_name)
            # train_label.append("No")
        else:
            print(folder_name)
        
        no.append(folder_name)


train_df =pd.DataFrame(train_file,columns=["pat_id","file_id","label_id"])
val_df = pd.DataFrame(val_file,columns=["pat_id","file_id","label_id"])
test_df = pd.DataFrame(test_file,columns=["pat_id","file_id","label_id"])
train_df.to_csv("D:/NewEra/" + 'M_train.csv', index=False)
val_df.to_csv("D:/NewEra/" + 'M_val.csv', index=False)
test_df.to_csv("D:/NewEra/" + 'M_test.csv', index=False)

In [9]:
from torch.utils.data import Dataset
import cv2
import pandas as pd




class testDataset(Dataset):

    def __init__(self,split,transform=None):
        super().__init__()
        self.path = r"D:/NewEra"
        self.transform = transform
        self.split = split
        #self.df_N = pd.read_csv(self.path + '/N_'+split+'.csv')
        self.df_M = pd.read_csv(self.path + '/M_'+split+'.csv')
        #self.df_no = pd.read_csv(self.path + '/No_'+split+'.csv')
        self.df = pd.concat([self.df_M],ignore_index=True) # 행 번호를 다시 0부터 쭈욱
        self.df = self.df.sample(frac=1).reset_index(drop=True)
        del(self.df_M)

    def __len__(self):
        return len(self.df)

    def get_img(self,pat_id,file_id,label_id,patch_id=None):
        if label_id == 'M':
            image = cv2.imread("{}/M/{}_{}.png".format(self.path, pat_id, file_id), cv2.IMREAD_COLOR)
            image = self.transform(image)
            label = 0
            return image, label

        elif label_id == "N":
            image = cv2.imread("{}/N/{}_{}.png".format(self.path , pat_id, file_id), cv2.IMREAD_COLOR)
            image= self.transform(image)
            label = 1
            return image, label

        else :
            image = cv2.imread("{}/No/{}_{}.png".format(self.path , pat_id, file_id), cv2.IMREAD_COLOR)
            image= self.transform(image)
            label = 2
            return image, label
        

    def __getitem__(self, idx):
        pat_id, file_id, label_id = self.df.iloc[idx,:]
        images, labels = self.get_img(pat_id, file_id,label_id)

        return images, labels
realtest_set = testDataset('realtest',transformations)
realtest_loader = DataLoader(realtest_set, batch_size = 3, shuffle = True)

AttributeError: 'testDataset' object has no attribute 'df'

In [ ]:
ckpt = torch.load(r'ckpt\Arts_best_checkpoint.pth.tar')
model.load_state_dict(ckpt['state_dict'])

In [ ]:
model.eval()
with torch.no_grad():
    total_loss=0
    pred=[]
    real=[]
    output_decoders = []
    for i, (data,label) in enumerate(test_loader):
        img = data.cuda()
        label = label.cuda()

        confidence,encoder,decoder = model(img)

        pred += confidence.argmax(dim=1).cpu().tolist()
        real += label.cpu().tolist()

        output_decoders.append(decoder)

In [ ]:
from sklearn.metrics import classification_report

target_names = ['Malignant', 'Normal', 'Noise']
print(classification_report(real, pred, target_names=target_names))
print("Accuarcy score : ", accuracy_score(real, pred))
print("f1 score : ", f1_score(real, pred, average='macro'))

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
X = output_decoders.toarray()
X_embedded = TSNE(n_components=2, learning_rate='auto',
                   init='random', perplexity=3).fit_transform(X)
tsne_df = pd.DataFrame(X_embedded, columns = ['component 0', 'component 1'])



In [ ]:
import matplotlib.pyplot as plt

# class target 정보 불러오기 
tsne_df['target'] = df['target']

# target 별 분리
tsne_df_0 = tsne_df[tsne_df['target'] == 0]
tsne_df_1 = tsne_df[tsne_df['target'] == 1]
tsne_df_2 = tsne_df[tsne_df['target'] == 2]

# target 별 시각화
plt.scatter(tsne_df_0['component 0'], tsne_df_0['component 1'], color = 'pink', label = 'setosa')
plt.scatter(tsne_df_1['component 0'], tsne_df_1['component 1'], color = 'purple', label = 'versicolor')
plt.scatter(tsne_df_2['component 0'], tsne_df_2['component 1'], color = 'yellow', label = 'virginica')

plt.xlabel('component 0')
plt.ylabel('component 1')
plt.legend()
plt.show()

In [ ]:
import torch.nn as nn
encoder22= nn.Sequential(
    nn.Conv2d(3, 32, 3, stride=3, padding=1),  # b, 3, 10, 10
    nn.ReLU(True),
# b, 16, 5, 5
    nn.Conv2d(32, 32, 3, stride=3, padding=1),  # b, 8, 3, 3
    nn.ReLU(True),

    nn.Conv2d(32, 8, 3, stride=3, padding=1),  # b, 8, 3, 3
    nn.ReLU(True),
    nn.Flatten(),
    nn.Linear(11552,2)
    )# b, 8, 2, 2
    

decoder22 = nn.Sequential(
            
            
    nn.ConvTranspose2d(8, 32, 5, stride=3, padding=1),  # b, 16, 5, 5
    nn.ReLU(True),
    nn.ConvTranspose2d(32, 32, 5, stride=3, padding=0),  # b, 8, 15, 15
    nn.ReLU(True),
    nn.ConvTranspose2d(32, 3, 3, stride=3, padding=4),  # b, 1, 28, 28
    nn.Tanh())

x= torch.rand(2,3,1024,1024)
y = torch.rand(2,2)
En = encoder22(x)
linear = nn.Linear(2,11552)
En = linear(En)
En = En.view(-1,8,38,38)
de = decoder22(En )
print(En.size(),de.size())